In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
from branca.colormap import LinearColormap

In [2]:
df = pd.read_csv("map_source.csv")
df.columns

Index(['Unnamed: 0', 'MIN_PRICE', 'MAX_PRICE', 'CLASSIFICATION_GENRE',
       'VENUE_LATITUDE', 'VENUE_LONGITUDE'],
      dtype='object')

In [3]:


# Assuming your DataFrame is named 'df' with columns 'latitude', 'longitude', and 'price'

# Create a map centered on the mean latitude and longitude of your data points
m = folium.Map(location=[df['VENUE_LATITUDE'].mean(), df['VENUE_LONGITUDE'].mean()], zoom_start=6)

# Create a color map
quartile1 = df["MAX_PRICE"].quantile(0.25)
quartile3 = df["MAX_PRICE"].quantile(0.75)
iqr = quartile3 - quartile1
lower_bound = quartile1 - iqr * 1.5
upper_bound = quartile3 + iqr * 1.5
no_outliers = df["MAX_PRICE"][(df["MAX_PRICE"] >= lower_bound) & (df["MAX_PRICE"] <= upper_bound)]
color_map = LinearColormap(colors=['green', 'yellow', 'red'], vmin=no_outliers.min(), vmax=no_outliers.max())

# Add points to the map
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['VENUE_LATITUDE'], row['VENUE_LONGITUDE']],
        radius=5,
        popup=f"Price: {row['MAX_PRICE']}",
        color=color_map(row['MAX_PRICE']),
        fill=True,
        fillColor=color_map(row['MAX_PRICE']),
        fillOpacity=0.7
    ).add_to(m)

# Add color map to the map
color_map.add_to(m)

# Save the map
m.save("price_map.html")


In [4]:
df["CLASSIFICATION_GENRE"].unique()

array(['World', 'Hip-Hop/Rap', 'Pop', 'Undefined', 'Religious', 'Other',
       'Rock', 'Dance/Electronic', 'Alternative', 'Reggae', 'Metal',
       'Country', 'R&B', 'Blues', 'Latin', 'Jazz', 'Folk', 'Classical',
       "Children's Music", 'New Age'], dtype=object)

In [7]:
import random
m_genre = folium.Map(location=[df['VENUE_LATITUDE'].mean(), df['VENUE_LONGITUDE'].mean()], zoom_start=6)
# Generate a list of 20 distinct colors
colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(20)]
color_map = dict(zip(df["CLASSIFICATION_GENRE"].unique(), colors))

for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['VENUE_LATITUDE'], row['VENUE_LONGITUDE']],
        radius=5,
        popup=f"Genre: {row['CLASSIFICATION_GENRE']}",
        color=color_map[row['CLASSIFICATION_GENRE']],
        fill=True,
        fillColor=color_map[row['CLASSIFICATION_GENRE']],
        fillOpacity=0.7
    ).add_to(m_genre)

# Create a legend
legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 320px; 
    border:2px solid grey; z-index:9999; font-size:14px; overflow-y: auto;
    background-color:white;
    ">
    <div style="position: relative; top: 5px; left: 5px;">
    <h4>Genre Legend</h4>
    '''

for genre, color in color_map.items():
    legend_html += f'<p><span style="background-color:{color};opacity:0.7;margin-right:5px;">&nbsp;&nbsp;&nbsp;&nbsp;</span>{genre}</p>'

legend_html += '''
    </div>
    </div>
    '''

m_genre.get_root().html.add_child(folium.Element(legend_html))


m_genre.save("genre_map.html")